In [8]:
import numpy as np
import sympy as sp

from functools import partial
from composipy.optimize import minimize_panel_weight, calc_lp_proportions
from composipy import  OrthotropicMaterial, LaminateProperty
from composipy.nastranapi import build_pcomp

from generate_sequence import get_best_laminate_random

## Defining minimize function

In [75]:
constraint_SSCC = { 'x0' : ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
                'xa' : ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
                'y0' : ['TX', 'TY', 'TZ'],
                'yb' : ['TX', 'TY', 'TZ']
                }



panel = {'E1': 127560,
            'E2': 13030,
            'G12': 6410,
            'v12': 0.3,
            'a': 1181.1,
            'b': 746.74,
            'm': 5,
            'n': 5,
            'panel_constraint' : "PINNED",
}

panel_SSCC = {'E1': 127560,
            'E2': 13030,
            'G12': 6410,
            'v12': 0.3,
            'a': 1181.1,
            'b': 746.74,
            'm': 5,
            'n': 5,
            'panel_constraint' : constraint_SSCC,
}




minimize = partial(minimize_panel_weight, **panel)
minimize_SSCC = partial(minimize_panel_weight, **panel_SSCC)

In [72]:

Nxx, Nyy, Nxy = -2937.695, -581.4551094, 29.184305

res = minimize(Nxx = Nxx, Nyy=Nyy, Nxy=Nxy, plot=False)
res

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 7840.168892736888
       x: [ 1.987e+01 -5.050e-05 -1.000e+00]
     nit: 21
     jac: [ 1.184e+03  0.000e+00  0.000e+00]
    nfev: 108
    njev: 17

In [63]:
2.074e+01/0.127

163.30708661417322

In [68]:
2.057e+01/0.127

161.96850393700788

In [73]:
1.987e+01/0.127

156.45669291338584

In [5]:
p = calc_lp_proportions(res.x[1], res.x[2])
p

b = -1.0001002293030665
xi1_ = -1.0000501146515333


{'p0': -0.0008352499578162297,
 'p90': 5.334542833587676,
 'p45': 94.66629241637014}

In [6]:
p

{'p0': -0.0008352499578162297,
 'p90': 5.334542833587676,
 'p45': 94.66629241637014}

## Batch optimization

In [7]:
def transform_plies(n):

    n = int(n)
    if n < 2:
        return int(0)
    elif n % 2:
        return n + 1
    else:
        return n + 2

In [103]:
import pandas as pd


loads = r'E:\Rafael-Documentos-trabalhos\PERIODOS\ITA-2019--\0 - TESE\_works\box_structural_optimization\11th trial SSCC\loads\loads_3.csv'
loads_results = r'E:\Rafael-Documentos-trabalhos\PERIODOS\ITA-2019--\0 - TESE\_works\box_structural_optimization\11th trial SSCC\loads\loads_3_results.xlsx'
pcomp_file = r'E:\Rafael-Documentos-trabalhos\PERIODOS\ITA-2019--\0 - TESE\_works\box_structural_optimization\11th trial SSCC\loads\pcomp_3.dat'
loads_df = pd.read_csv(loads, sep=';')

optimization_result = {}
optimization_result['Nxx'] = []
optimization_result['Nyy'] = []
optimization_result['Nxy'] = []
optimization_result['t'] = []
optimization_result['xiD1'] = []
optimization_result['xiD3'] = []
optimization_result['message'] = []
optimization_result['nfev'] = []
optimization_result['n_plies'] = []
optimization_result['stack'] = []

pcomp = ''

panels = [(10, 'pinned'), (11, 'pinned'), (12, 'pinned'), (13, 'pinned'), (14, 'pinned'),
           (15, 'pinned'), (16, 'pinned'), (17, 'pinned'), (18, 'pinned')]

panels = [(10, 'sscc'), (11, 'pinned'), (12, 'pinned'), (13, 'pinned'), (14, 'pinned'),
           (15, 'sscc'), (16, 'sscc'), (17, 'pinned'), (18, 'pinned')]



for i, p_bc in enumerate(panels):

    p, bc = p_bc
    Nxx_ = loads_df.iloc[i][0]
    Nyy_ = loads_df.iloc[i][1]
    Nxy_ = loads_df.iloc[i][2]

    print(f'====== i => {i} ======= ')
    print(f'BC = {bc}')
    if bc == 'pinned':
        res = minimize(Nxx=Nxx_, Nyy=Nyy_, Nxy=Nxy_)   
    elif bc == 'sscc':
        res = minimize_SSCC(Nxx=Nxx_, Nyy=Nyy_, Nxy=Nxy_)   
    print(res)

    n_plies = int(res.x[[0]]/0.127 + 0.99)
    stack = get_best_laminate_random(float(res.x[[1]]), float(res.x[[2]]), n_plies, n_sequences=500)


    optimization_result['Nxx'].append(Nxx_)
    optimization_result['Nyy'].append(Nyy_) 
    optimization_result['Nxy'].append(Nxy_)
    optimization_result['t'].append(float(res.x[[0]]))
    optimization_result['xiD1'].append(float(res.x[[1]]))
    optimization_result['xiD3'].append(float(res.x[[2]]))
    optimization_result['message'].append(res.message)
    optimization_result['nfev'].append(res.nfev)
    optimization_result['n_plies'].append(n_plies)
    optimization_result['stack'].append(str(stack))
    
    pcomp += build_pcomp(sequence=stack, midi=1, ti=0.127, pid=p) + '\n'



====== i => 0 ======= 
BC = sscc
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 6967.9107209889235
       x: [ 1.910e+01 -5.000e-05 -1.000e+00]
     nit: 12
     jac: [ 1.094e+03  0.000e+00  0.000e+00]
    nfev: 38
    njev: 8
0.5869916823130004
====== i => 1 ======= 
BC = pinned
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3598.532018909367
       x: [ 1.532e+01  5.000e-05 -1.000e+00]
     nit: 6
     jac: [ 7.045e+02  0.000e+00  0.000e+00]
    nfev: 23
    njev: 5
0.4926864405565011
====== i => 2 ======= 
BC = pinned
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1065.566542397302
       x: [ 1.021e+01  5.000e-05 -1.000e+00]
     nit: 12
     jac: [ 3.130e+02  0.000e+00  0.000e+00]
    nfev: 34
    njev: 8
0.4213088235380392
====== i => 3 ======= 
BC = pinned


C:\Users\Pichau\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1095.1508217682265
       x: [ 1.031e+01  5.000e-05 -1.000e+00]
     nit: 14
     jac: [ 3.187e+02  0.000e+00  0.000e+00]
    nfev: 54
    njev: 10
0.4545753588896514
====== i => 4 ======= 
BC = pinned
 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 3816.537895315506
       x: [ 1.563e+01 -6.145e-02 -8.772e-01]
     nit: 39
     jac: [ 7.327e+02  0.000e+00  0.000e+00]
    nfev: 264
    njev: 35
0.4468952066585032
====== i => 5 ======= 
BC = sscc
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 7895.387381756973
       x: [ 1.991e+01 -5.025e-05 -1.000e+00]
     nit: 13
     jac: [ 1.190e+03  0.000e+00  0.000e+00]
    nfev: 39
    njev: 9
0.6464402169259242
====== i => 6 ======= 
BC = sscc
 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 7785.567212079335
       x: [ 1.982e+01 -5

In [104]:
optimization_result_df = pd.DataFrame(optimization_result)
optimization_result_df

,Nxx,Nyy,Nxy,t,xiD1,xiD3,message,nfev,n_plies,stack
0,-2490.364625,-572.743225,287.530987,19.100036,-0.000050,-1.000000,Optimization terminated successfully,38,151,"[0, -45, -45, 45, 0, 45, -45, -45, -45, 45, 0,..."
1,-1532.191125,-45.652111,-36.843590,15.324105,0.000050,-1.000000,Optimization terminated successfully,23,121,"[45, 45, 45, 90, -45, 45, -45, 90, -45, 0, 0, ..."
2,-467.461262,12.969216,-62.925180,10.213945,0.000050,-1.000000,Optimization terminated successfully,34,81,"[45, 45, 90, 45, -45, -45, 45, 45, 45, 90, -45..."
3,-462.645900,21.729247,-190.957012,10.307610,0.000050,-1.000000,Optimization terminated successfully,54,82,"[-45, -45, 45, 45, -45, -45, 45, 45, 0, -45, 4..."
4,-1515.318500,-122.926700,-200.959900,15.627513,-0.061448,-0.877203,Positive directional derivative for linesearch,264,124,"[-45, 45, -45, 45, 45, 0, 45, -45, -45, -45, 4..."
5,-2697.168750,-719.800825,-132.325100,19.912440,-0.000050,-1.000000,Optimization terminated successfully,39,157,"[45, 90, -45, -45, -45, 45, 45, -45, 45, 0, 90..."
6,-2868.876500,-602.723438,30.149411,19.819685,-0.000050,-1.000000,Positive directional derivative for linesearch,34,157,"[-45, 45, 45, -45, 0, 0, -45, 90, 45, -45, 0, ..."
7,-1614.706875,-33.143195,-322.062412,15.632314,0.000050,-1.000000,Positive directional derivative for linesearch,30,124,"[-45, 45, -45, -45, -45, 45, -45, 45, 45, 90, ..."
8,-499.660200,-3.730002,-285.643175,10.841694,0.000050,-1.000000,Optimization terminated successfully,65,86,"[0, -45, 45, -45, 45, -45, 45, 45, 45, 90, -45..."


In [105]:
optimization_result_df.to_excel(loads_results)

with open(pcomp_file, 'w') as f:
    f.write(pcomp)

## Sanity check

In [22]:
from composipy import OrthotropicMaterial, LaminateProperty, PlateStructure

In [23]:
panel

{'E1': 127560,
 'E2': 13030,
 'G12': 6410,
 'v12': 0.3,
 'a': 1181.1,
 'b': 746.74,
 'm': 5,
 'n': 5,
 'panel_constraint': 'PINNED'}

In [24]:
mat = OrthotropicMaterial(127560, 13030, 0.3, 6410, 0.127)

In [55]:
124/4   

31.0

In [59]:
stack = [90] * 0 + [45, -45] * 22
stack += stack[::-1]

In [60]:
len(stack)

88

In [61]:
Nxx, Nyy, Nxy = -521.6847188,	30.37621438,	-335.3535375

lam = LaminateProperty(stack, mat)
struct = PlateStructure(lam, 1181.1, 746.74, "PINNED", Nxx=Nxx, Nyy=Nyy, Nxy=Nxy, m=12, n=12)
struct.buckling_analysis()

(array([1.07352466, 1.20327298, 1.3992801 , 1.76587299, 2.34677505]),
 array([[-2.22816564e-01, -3.31319936e-02,  2.51445868e-01,
          2.52595448e-01,  2.45706544e-01],
        [ 3.40393963e-01, -2.47967345e-01, -4.99893802e-01,
         -2.19511824e-01, -2.21071745e-01],
        [-1.75912153e-01,  1.28220562e-02,  2.05560477e-01,
          1.80772479e-01,  1.67785112e-01],
        [-9.55772322e-02,  2.19417129e-01,  1.84900415e-01,
         -4.34109743e-02, -5.30805792e-02],
        [ 2.48039739e-02,  6.37267870e-02,  3.10498088e-04,
         -5.98927904e-02, -4.13271042e-02],
        [-6.94218179e-02,  2.71813548e-02,  1.00151863e-01,
          6.04317727e-02,  6.99837206e-02],
        [-1.52919850e-02,  2.77636870e-02,  2.57856723e-02,
         -7.05468751e-03, -1.04422369e-02],
        [-4.29405698e-02,  1.72933140e-02,  6.75175159e-02,
          4.18393551e-02,  4.60973435e-02],
        [-7.92604453e-03,  1.47998652e-02,  1.58351465e-02,
         -1.53603546e-03, -2.44858710e

In [45]:
lam.xiD

array([-2.29745370e-01,  1.75057870e-02, -5.40509259e-01, -5.41275459e-17])